In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/dataset')
print(os.getcwd())

/content/drive/MyDrive/dataset


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd

In [6]:
data = pd.read_csv('./dfdfdf.csv')
data = data[['Unnamed: 0','keyword_score','location_score','text','target']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7552 entries, 0 to 7551
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      7552 non-null   int64  
 1   keyword_score   7552 non-null   float64
 2   location_score  7552 non-null   float64
 3   text            7552 non-null   object 
 4   target          7552 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 295.1+ KB


In [ ]:
data.describe()

In [7]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import torch


# 1. 트위터 특화 BERT 로드
model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. 문장 리스트를 CLS 임베딩으로 벡터화하는 함수
def get_cls_batch(texts, batch_size=16, max_len=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    all_embeddings = []
    dataloader = DataLoader(texts, batch_size=batch_size)

    for batch in dataloader:
        # 트윗 전처리
        batch = [text.replace("http://", "").replace("https://", "").strip() for text in batch]

        # 토크나이징
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
        all_embeddings.append(cls_embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [8]:
vector1 = get_cls_batch(data['text'].tolist())

In [9]:
vector1

array([[-0.0160815 ,  0.14079247, -0.03751713, ..., -0.29151142,
        -0.08364729, -0.0158129 ],
       [-0.034004  ,  0.15643029, -0.02315493, ..., -0.12306041,
        -0.01378192,  0.01959843],
       [ 0.02784241,  0.14669035, -0.02050884, ..., -0.15256435,
        -0.03087958,  0.05618834],
       ...,
       [-0.01305907,  0.12220544, -0.01881577, ..., -0.31067052,
        -0.0349055 , -0.00895749],
       [-0.00975733,  0.06710597, -0.02352173, ..., -0.33086678,
        -0.00649847,  0.02098333],
       [ 0.01588063,  0.17976163,  0.01136754, ..., -0.18811521,
        -0.07754089,  0.02618206]], dtype=float32)

In [10]:
v1 = data['keyword_score'].to_numpy().reshape(-1, 1)
v2 = data['location_score'].to_numpy().reshape(-1, 1)

In [11]:
x=pd.DataFrame(np.concatenate((vector1,v1,v2),axis=1))
x.columns = x.columns.astype(str)

In [14]:
y=data['target']

In [15]:
x.shape, y.shape

((7552, 770), (7552,))

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=123)
x_train, x_val,y_train, y_val=train_test_split(x_train,y_train,test_size=0.15,random_state=123)

In [17]:
from sklearn.metrics import f1_score

In [18]:
x_train.columns = x_train.columns.map(str)
x_test.columns = x_test.columns.map(str)
x_val.columns = x_val.columns.map(str)

In [24]:
from sklearn.ensemble import RandomForestClassifier
RFmodel= RandomForestClassifier().fit(x_train, y_train)

y_pred = RFmodel.predict(x_test)

train_score=RFmodel.score(x_train,y_train)
validation_score=RFmodel.score(x_val,y_val)

print(train_score)
print(validation_score)

differ=round((train_score-validation_score),3)

if not differ<=0.05:
    print(differ,'=> 과적합')
else:
    print(differ,'=> 과적합 아님')

print('f1 score =',round(f1_score(y_test,y_pred),3))

0.9988871578010238
0.8726355611601513
0.126 => 과적합
f1 score = 0.824


In [20]:
from sklearn.svm import SVC

SVMmodel = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    probability=True,
    random_state=123
).fit(x_train, y_train)

y_pred = SVMmodel.predict(x_test)

train_score=SVMmodel.score(x_train, y_train)
validation_score=SVMmodel.score(x_val, y_val)

print(train_score)
print(validation_score)

differ=round((train_score-validation_score), 3)

if not differ <= 0.05:
    print(differ, '=> 과적합')
else:
    print(differ, '=> 과적합 아님')

print('f1 score =', round(f1_score(y_test, y_pred), 3))

0.9111951925217004
0.8877679697351829
0.023 => 과적합 아님
f1 score = 0.875


In [21]:
x_train = x_train.astype(float)
x_val = x_val.astype(float)
x_test = x_test.astype(float)

In [22]:
from xgboost import XGBClassifier

XGBmodel = XGBClassifier(
    n_estimators=900,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=123
).fit(x_train, y_train)

y_pred = XGBmodel.predict(x_test)

train_score=XGBmodel.score(x_train, y_train)
validation_score=XGBmodel.score(x_val, y_val)

print(train_score)
print(validation_score)

differ=round((train_score-validation_score), 3)

if not differ <= 0.05:
    print(differ, '=> 과적합')
else:
    print(differ, '=> 과적합 아님')

print('f1 score =', round(f1_score(y_test, y_pred), 3))

0.9988871578010238
0.8965952080706179
0.102 => 과적합
f1 score = 0.879


In [25]:
# Stacking ensemble(스태킹 앙상)
from IPython.display import Image, display

url = "https://miro.medium.com/v2/resize:fit:786/format:webp/1*DM1DhgvG3UCEZTF-Ev5Q-A.png"
display(Image(url=url))

#개별 모델(base model)의 예측값을 최종모델(meta model)의 학습 데이터로 사용하여 최종 예측을 수행하는 방법

In [34]:
#별칭과 실제 모델을 (str,estimator)형태
base_models = [
    ('rf', RFmodel),
    ('svm',SVMmodel),
    ('xgb',XGBmodel)
]

In [33]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# 메타 모델은 과적합 줄이기 위해 단순한 로지스틱 회귀 사용
meta_model = LogisticRegression(max_iter=1000, class_weight='balanced')

# Stratified KFold로 클래스 비율 유지
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

# StackingClassifier 구성
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=cv,
    passthrough=True,
    n_jobs=-1
).fit(x_train, y_train)

y_pred5 = stack_model.predict(x_test)

train_score=stack_model.score(x_train,y_train)
validation_score=stack_model.score(x_val,y_val)

print(train_score)
print(validation_score)

differ=round((train_score-validation_score),3)

if not differ<=0.05:
    print(differ,'=> 과적합')
else:
    print(differ,'=> 과적합 아님')

print('f1 score =', round(f1_score(y_test, y_pred5), 3))

0.9583797017582907
0.8953341740226987
0.063 => 과적합
f1 score = 0.889
